In [1]:
from tool import *

# 1. 재작업비용 내역 DB 불러오기

In [23]:
with open('D:/Data/nicepeople_cost_db.bin', 'rb') as f:
    rw_df = pickle.load(f)

# 2. 나이스피플 청구서 불러와서 데이타 cleaning

In [24]:
vendor = 'Pegatron'

In [25]:
fn = get_filename()
df = pd.read_excel(fn, sheet_name='업무대행비')
i = df.loc[df['Unnamed: 1'].str.contains('작업일', na=False)].index[0]
df = pd.read_excel(fn, skiprows=i+1, sheet_name='업무대행비')
df = df.iloc[:, 1:]

In [26]:
df.head()

,작업일,Unnamed: 2,인원 수,평일,특근(휴일),잔업1TH= ((일당/8)* 1.5),Unnamed: 7,Unnamed: 8,소계
0,NaN,NaN,NaN,134561.0,201842.0,시간TH,인원수,25230.0,NaN
1,2023-03-01 00:00:00,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
2,2023-03-02 00:00:00,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
3,2023-03-03 00:00:00,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
4,2023-03-04 00:00:00,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0


In [27]:
df.columns = [i.strip() for i in df.columns.tolist()]
price_table = {'normal':df.loc[0, '평일'], 'holiday':df.loc[0, '특근(휴일)'], 'overtime':df.loc[0, 'Unnamed: 8']}
df.columns = df.iloc[0]
df = df.drop(0)

In [28]:
df.head()

,NaN,NaN,NaN,134561.0,201842.0,시간TH,인원수,25230.0,NaN
1,2023-03-01 00:00:00,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
2,2023-03-02 00:00:00,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
3,2023-03-03 00:00:00,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
4,2023-03-04 00:00:00,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
5,2023-03-05 00:00:00,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0


In [29]:
df.columns = ['Inspection Date', '삭제1', 'Worker1', 'Amount_Normal', 'Amount_Holiday', 'Overtime', 'Worker2', 'Amount_Overtime', 'Total_Amount']

In [30]:
df = df.drop(['삭제1'], axis=1)

In [31]:
df = df[df['Inspection Date'].notnull()]
df = df[df['Inspection Date'].str.contains('2022-', na=True)]
df = df[df['Worker1'] > 0]

In [32]:
df

,Inspection Date,Worker1,Amount_Normal,Amount_Holiday,Overtime,Worker2,Amount_Overtime,Total_Amount
30,2023-03-30 00:00:00,5.0,672805.0,NaN,NaN,NaN,NaN,672805.0


In [33]:
for k, v in price_table.items():
    df[k] = v
df = df.convert_dtypes()
df = df.fillna(0)
df['Weekday'] = df['Inspection Date'].dt.weekday
cond = df['Weekday'] < 5
df.loc[cond, 'Amount_Normal_Cal'] = df.loc[cond, 'Worker1'] * df.loc[cond, 'normal']
df['Amount_Normal_Cal'].fillna(0, inplace=True)
df.loc[~cond, 'Amount_Holiday_Cal'] = df.loc[~cond, 'Worker1'] * df.loc[~cond, 'holiday']
df['Amount_Holiday_Cal'].fillna(0, inplace=True)
cond2 = df['Overtime'] > 0
df.loc[cond2, 'Amount_Overtime_cal'] = df.loc[cond2, 'Overtime'] * df.loc[cond2, 'Worker2'] * df.loc[cond2, 'overtime']
df['Amount_Overtime_cal'].fillna(0, inplace=True)
df['Total_Amount_cal'] = df['Amount_Normal_Cal'] + df['Amount_Holiday_Cal'] + df['Amount_Overtime_cal']
df['Total_Amount_cal'].fillna(0, inplace=True)
df['Amount_Check'] = (df['Total_Amount'] == df['Total_Amount_cal'])

In [34]:
df

,Inspection Date,Worker1,Amount_Normal,Amount_Holiday,Overtime,Worker2,Amount_Overtime,Total_Amount,normal,holiday,overtime,Weekday,Amount_Normal_Cal,Amount_Holiday_Cal,Amount_Overtime_cal,Total_Amount_cal,Amount_Check
30,2023-03-30,5,672805,0,0,0,0,672805,134561,201842,25230,3,672805,0,0,672805,True


In [35]:
# df['Total_Amount'] = df['Total_Amount_cal'] # 비교하여 정확한 값을  Total Amount로 함
df = df[['Inspection Date', 'Worker1', 'Amount_Normal', 'Amount_Holiday', 'Overtime', 'Worker2', 'Amount_Overtime',
         'Total_Amount', 'normal', 'holiday', 'overtime']]
df['Amount_with_VAT'] = df['Total_Amount']*1.1
df['Amount_with_VAT'] = df['Amount_with_VAT'].round().astype(int)
df.insert(1, 'Supplier', vendor)
df.reset_index(drop=True, inplace=True)

C:\Users\paul76.lee\AppData\Local\Temp/ipykernel_15564/1587937323.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Amount_with_VAT'] = df['Total_Amount']*1.1
C:\Users\paul76.lee\AppData\Local\Temp/ipykernel_15564/1587937323.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Amount_with_VAT'] = df['Amount_with_VAT'].round().astype(int)


In [36]:
df['Amount_with_VAT'].sum() # 해당월 총 재작업 비용

740086

In [37]:
df

,Inspection Date,Supplier,Worker1,Amount_Normal,Amount_Holiday,Overtime,Worker2,Amount_Overtime,Total_Amount,normal,holiday,overtime,Amount_with_VAT
0,2023-03-30,Pegatron,5,672805,0,0,0,0,672805,134561,201842,25230,740086


In [38]:
rw_df.shape

(96, 13)

In [39]:
rw_df = pd.concat([rw_df, df])

In [40]:
rw_df.reset_index(drop=True, inplace=True)

# 3. 데이타 저장

In [41]:
rw_df.shape # 파일 저장 전 다시 한번 확인하자!

(97, 13)

In [42]:
with open('D:/Data/nicepeople_cost_db.bin', 'wb') as f:
    pickle.dump(rw_df, f)

In [43]:
with open('D:/Data/DB backup/nicepeople_cost_db.bin', 'wb') as f:
    pickle.dump(rw_df, f)